In [14]:
import numpy as np
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, LSTM
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, MaxPool1D, GaussianNoise, GlobalMaxPooling1D
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sn
import os

Load in the preprocessed data

In [3]:
from save_utils import load_sliced_numpy_array

melspec_data = load_sliced_numpy_array('melspec_features')
labels = np.load('data/labels.npy')

print("Features:", melspec_data.shape)
print("Labels:", labels.shape)

Loaded 12 files:
    melspec_features_001.npy
    melspec_features_002.npy
    melspec_features_003.npy
    melspec_features_007.npy
    melspec_features_012.npy
    melspec_features_006.npy
    melspec_features_010.npy
    melspec_features_004.npy
    melspec_features_005.npy
    melspec_features_011.npy
    melspec_features_008.npy
    melspec_features_009.npy
Features: (4068, 33088)
Labels: (4068, 5)


Split the data into train and test set

In [19]:
mel_train, mel_test, lab_train, lab_test = train_test_split(melspec_data, labels, test_size=0.2, random_state=42)

assert mel_train.shape[0] == lab_train.shape[0] and mel_test.shape[0] == lab_test.shape[0]
assert mel_train.shape[1] == mel_test.shape[1] and lab_train.shape[1] == lab_test.shape[1]

print(mel_train.shape)
print(mel_test.shape)

print(lab_train.shape)
print(lab_test.shape)


(3254, 33088)
(814, 33088)
(3254, 5)
(814, 5)


Normalize the data (3254, 64, 517)

In [7]:
def normalization(mel_train, mel_test):
    maximum = np.amax(mel_train)
    mel_train = mel_train/maximum
    mel_test = mel_test/maximum
    return (mel_train.astype(np.float32), mel_test.astype(np.float32))

mel_train_n, mel_test_n = normalization(mel_train, mel_test)

Reshape the melspec the models dimensions

In [8]:
n_mels = 64
# .reshape(n_mels, -1)

# Reshaping Mel-Spectrogram
def reshape_melspectogram(mel_train, mel_test):
    n, m = mel_train.shape
    mel_train = mel_train.reshape((n, n_mels, int(m/n_mels), 1))
    n, m = mel_test.shape
    mel_test = mel_test.reshape((n, n_mels, int(m/n_mels), 1))
    return mel_train, mel_test

mel_train_n, mel_test_n = reshape_melspectogram(mel_train_n, mel_test_n)

print(mel_train_n.shape)
print(mel_test_n.shape)

(3254, 64, 517, 1)
(814, 64, 517, 1)


Define the shot-chunk-cnn model

In [9]:
# Define the model
def shortchunckcnn(input_shape, output_shape):
    model = Sequential()
    model.add(Conv2D(8, (3,3), activation= 'relu', input_shape=input_shape, padding= 'same'))
    model.add(MaxPooling2D((4,4), padding= 'same'))
    model.add(Conv2D(16, (3,3), activation= 'relu', padding= 'same'))
    model.add(MaxPooling2D((4,4), padding= 'same'))
    model.add(Conv2D(32, (3,3), activation= 'relu', padding= 'same'))
    model.add(MaxPooling2D((4,4), padding= 'same'))
    model.add(Conv2D(64, (3,3), activation= 'relu', padding= 'same'))
    model.add(MaxPooling2D((4,4), padding= 'same'))
    model.add(Conv2D(64, (3,3), activation= 'relu', padding= 'same'))
    model.add(MaxPooling2D((4,4), padding= 'same'))
    model.add(Flatten())
    model.add(Dense(32, activation= 'relu'))
    model.add(Dense(output_shape, activation= 'softmax'))

    model.compile(optimizer= 'Adam', loss= 'categorical_crossentropy')
    model.summary()

    return model

model = shortchunckcnn(mel_train_n[0].shape, labels.shape[1])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 517, 8)        80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 130, 8)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 130, 16)       1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 33, 16)        0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 33, 32)         4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 9, 32)         0

2022-12-20 15:12:00.598689: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


_________________________________________________________________


Train the defined model

In [10]:
checkpoint = ModelCheckpoint("models/short_chunk_cnn_{epoch:03d}.h5", save_freq=5)

epochs=3

model.fit(mel_train_n, lab_train, epochs=epochs, callbacks=[checkpoint], batch_size=32, verbose=1)

model.save("models/short_chunk_cnn_complete_{epochs}.h5".format(epochs=epochs))

Epoch 1/3
102/102 [==============================] - 11s 97ms/step - loss: 1.3587
Epoch 2/3
102/102 [==============================] - 10s 99ms/step - loss: 1.2571
Epoch 3/3
102/102 [==============================] - 10s 97ms/step - loss: 1.2479


In [11]:
model = load_model("models/short_chunk_cnn_complete_{epochs}.h5".format(epochs=epochs))

# Training Accuracy
y_pred = model.predict(mel_train_n)
y_pred = np.argmax(y_pred, axis= -1)
y_true = np.argmax(lab_train, axis= -1)

correct = len(y_pred) - np.count_nonzero(y_pred - y_true)
acc = correct/ len(y_pred)
acc = np.round(acc, 4) * 100

print(acc)

102/102 [==============================] - 3s 26ms/step
41.24


In [12]:

print("Train Accuracy: ", correct, "/", len(y_pred), " = ", acc, "%")

# Testing Accuracy
y_pred = model.predict(mel_test)
y_pred = np.argmax(y_pred, axis= -1)
y_true = np.argmax(y_test, axis= -1)

correct = len(y_pred) - np.count_nonzero(y_pred - y_true)
acc = correct/ len(y_pred)
acc = np.round(acc, 4) * 100
print("Testing Accuracy", acc)

class_names = ["Blues", "Classical", "Country", "Disco", "Hiphop", "Jazz", "Metal", "Pop", "Reggae", "Rock"]
conf_mat = confusion_matrix(y_true, y_pred, normalize= 'true')
conf_mat = np.round(conf_mat, 2)

conf_mat_df = pd.DataFrame(conf_mat, columns= class_names, index= class_names)

plt.figure(figsize = (10,7), dpi = 200)
sn.set(font_scale=1.4)
sn.heatmap(conf_mat_df, annot=True, annot_kws={"size": 16}) # font size
plt.tight_layout()
plt.savefig(os.getcwd() + "/ensemble_mel_conf_mat1.png")

Train Accuracy:  1342 / 3254  =  41.24 %


2022-12-20 15:12:51.839921: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops_fused_impl.h:757 : INVALID_ARGUMENT: convolution input must be 4-dimensional: [32,33088]


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/conv2d/Relu' defined at (most recent call last):
    File "/Users/mstuffer/.pyenv/versions/3.9.15/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/mstuffer/.pyenv/versions/3.9.15/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/mstuffer/.pyenv/versions/3.9.15/envs/ai-industry/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/mstuffer/.pyenv/versions/3.9.15/envs/ai-industry/lib/python3.9/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/Users/mstuffer/.pyenv/versions/3.9.15/envs/ai-industry/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/mstuffer/.pyenv/versions/3.9.15/envs/ai-industry/lib/python3.9/sit